In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import seaborn as sns

import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_table('../data/train.tsv', index_col='Date', parse_dates=True)
test_df = pd.read_table('../data/test.tsv', index_col='Date', parse_dates=True)
sample_sub = pd.read_csv('../data/sample_submit.csv')

# set type label
train_df['type'] = 'train'
test_df['type'] = 'test'

# all data
all_df = pd.concat([train_df, test_df], axis=0)

## 学習する特徴量を作成
#### __欠損値処理__
→なし

#### __特徴量の削除/追加__
ピアソン相関から、  
[削除]  
- 時間ごとの気温"T0"~"T23"を消す
- 時間ごとの風速"WSR0"~"WSR24"を消す
- 海面気圧の前日からの変化"SLP_"を消す  

[追加]
- 風速の標準偏差を追加する(最大風速よりも平均風速の方が相関が高いため)
- 気温の標準偏差を追加する(とりあえず追加)
  
計26個

#### __データ変換__
→とりあえず全部標準化

In [ ]:
import re

def eda(all_df):
    # データの追加,気温・風速の標準偏差
    #1時間ごとの気温・風速を取得
    T_data = [s for s in all_df if re.match('T[0-9]', s) or re.match('T[1-2][0-9]', s)]
    WSR_data = [s for s in all_df if re.match('WSR[0-9]', s) or re.match('WSR[1-2][0-9]', s)]
    print(T_data.std(axis=1))
    #行ごとの標準偏差を追加
    all_df['T_SD'] = T_data.std(axis=1)
    all_df['WSR_SD'] = WSR_data.std(axis=1)
    # データの削除,T0~T23, WSR0~WSR23
    all_df = [s for s in all_df if not re.match('(T|WSR)[0-9]', s) or re.match('(T|WSR)[1-2][0-9]', s)]
    # データの削除,SLP_
    all_df = all_df.drop(columns=['SLP_'])
    
    # データ変換（標準化）
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    all_df

## モデル作成